In [6]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

#tb.loc[tb['Instrument Classification (ISO 10962 CFI code)'].isin(['JFTXFP'])]
#tb.plot.hist(tb.groupby(['Trading platform']).Price.agg(['mean', max]))
#tb.describe()
#print(tb.shape)

df = pd.read_feather('C:/Users/asus/Documents/rts27.feather')

#Columns deemed unneccessary for this task
df2 = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)


In [7]:
df.isnull().sum()

id                                       0
bank_name                                0
legal_entity_name                        0
file_name                                0
date_of_trading_day                      0
instrument_identifier               496474
instrument_classification          7525372
order_request_quotes_received       134236
executed_transactions              3032834
total_value_of_transactions        4976761
order_request_quotes_can_wdrwn     1172550
order_request_quotes_modified      2140910
median_transaction_size            4207916
median_size_request_quotes         2156422
designated_market_makers           9578651
active                                   0
date_created                             0
source_instrument_desc             9489024
source_instrument_ccy              2143014
source_market_segment              5539694
smartie_instrument_id             13064739
systematic_internaliser_flag       9793955
liquidity_provider_flag           12623214
traded_in_n

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets, linear_model
from sklearn.decomposition import TruncatedSVD


from datetime import datetime

#Columns deemed unneccessary for this task
#df = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)


#Filters the code to FX forwards only
#df = df.loc[(df['source_instrument_desc']).astype(str).str.contains("Forward")]
df=df2
df = df.loc[(df['instrument_classification']).astype(str).str.contains("JF") | (df['source_instrument_desc']).astype(str).str.contains("Forward")]
#Drops the columns of which half of the fields have no data.
#df = df.dropna(axis=1, thresh=2000000)

        
# Select target
y = df['total_value_of_transactions']
# To keep things simple, we'll use only numerical predictors
dropped_df = df.drop(['total_value_of_transactions'], axis=1)
X = dropped_df

im = SimpleImputer(strategy='mean',missing_values=np.nan)
y = y.to_numpy().reshape(-1,1)
y = im.fit_transform(y)
y = pd.DataFrame(y)
print(y)

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

categorical_cols = [col for col in X.columns if X[col].dtype in ['object']]

# Select numerical columns
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
       
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

numerical_transformer = SimpleImputer(strategy='mean',missing_values=np.nan)

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant',missing_values=None, fill_value='missing_value')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

sc = StandardScaler()

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ], remainder='passthrough')

model = linear_model.LinearRegression()

#print(preprocessor.fit_transform(X_train, y_train))

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('reduce_dims', TruncatedSVD(n_components=5)),
                              ('scaler', sc),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)


# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

print('Coefficients: \n', model.coef_)


                    0
0        0.000000e+00
1        0.000000e+00
2        0.000000e+00
3        0.000000e+00
4        3.128493e+11
...               ...
5851704  0.000000e+00
5851705  0.000000e+00
5851706  8.284364e+05
5851707  0.000000e+00
5851708  0.000000e+00

[5851709 rows x 1 columns]
Coefficients: 
 [[ 2.06959893e+08  7.84980909e+12 -3.33109461e+11 -1.25847163e+11
   2.43714781e+12]]


In [ ]:
import seaborn as sns

print(preds.shape)
#print(preds)
print(mean_absolute_error(y_valid, preds))

# Set the width and height of the figure
plt.figure(figsize=(30,6))

# Add title
plt.title("Average Arrival Delay for Spirit Airlines Flights, by Month")

# Bar chart showing average arrival delay for Spirit Airlines flights by month
sns.barplot(x=y_valid[0], y=X_valid['executed_transactions'])
#sns.plot(x=X_train, y=y_train)~

# Add label for vertical axis
plt.ylabel("Transaction value")

(1170342, 1)
250516149439.59473


0.23.1
